# Random forest support vector machine classifier (RF-SVC)
## Vincent Buekers
Promotor: Prof. dr. Johan A.K. Suykens

Supervision: Yingyi Chen

In [58]:
import numpy as np
import pandas as pd

from sklearn import datasets, tree, svm, linear_model, preprocessing
from sklearn.model_selection import RandomizedSearchCV, StratifiedShuffleSplit

from statistics import mode, StatisticsError

from joblib import Parallel, delayed

## Partition input space into subsets 
Subsets are obtained from the leaf nodes of an extremely randomized tree. For purposes of theoretical consistency, only one candidate feature is selected from all d features using the option max_features = 1, yielding totally random trees.

Note: these are non-overlapping subsets due to the recursive branching mechanism in decision trees

In [54]:
def extra_partition(X_train,X_test, y_train,y_test, idx_train,idx_test):

    subsets = []
    
    # totally randomized tree (max_features=1)
    extra = tree.ExtraTreeClassifier(max_features=1,min_samples_leaf = int(np.sqrt(len(X_train))) )
    extra.fit(X_train,y_train)
    
    # obtain leaf indices the datapoints appear in
    leaf_idx_train, leaf_idx_test = extra.apply(X_train), extra.apply(X_test)
    
    # Keep track of observation indexes and prepare for pandas' .groupby
    leaf_idx_train = pd.DataFrame(leaf_idx_train, index=idx_train)
    leaf_idx_test = pd.DataFrame(leaf_idx_test, index=idx_test)
    
    # Group train and test observations by their leaf node
    groups_train = leaf_idx_train.groupby(leaf_idx_train[0],axis=0).groups
    groups_test = leaf_idx_test.groupby(leaf_idx_test[0],axis=0).groups
    
    # collect all data back into one array, sorted by original observation indexes
    X_train, X_test = np.c_[idx_train,X_train], np.c_[idx_test,X_test]
    y_train, y_test = np.c_[idx_train,y_train], np.c_[idx_test,y_test]
    X, y = np.r_[X_train,X_test], np.r_[y_train,y_test]
    X, y = X[np.argsort(X[:,0])], y[np.argsort(y[:,0])]
    X, y = np.delete(X, 0, 1), np.delete(y, 0, 1)
    
    # tree predictions (only test observations will be retrieved later on)
    preds_tree = extra.predict(X).reshape(-1,1)
    
    # Obtain train and test subsets created by the leaf node partitioning
    # iterables are a list of Int64index objects for the data in each leaf node
    for leaf_train, leaf_test in zip(list(groups_train.values()),list(groups_test.values())) :
        
        # subset the data
        X_train_sub, y_train_sub = X[leaf_train], y[leaf_train]
        X_test_sub, y_test_sub, y_tree = X[leaf_test] ,y[leaf_test], preds_tree[leaf_test]
        
        # original indexes of the observations appearing in this leaf
        train_indexes = np.array(leaf_train).reshape(-1,1)
        test_indexes = np.array(leaf_test).reshape(-1,1)
        
        # training subset including original observation indexes
        sub_train = np.concatenate((train_indexes, X_train_sub, y_train_sub), axis=1)
        # testing subset including original observation indexes and tree predictions
        sub_test = np.concatenate((test_indexes, X_test_sub, y_test_sub, y_tree), axis=1)

        subsets.append([sub_train,sub_test])
    
    return subsets

# Embedded SVM classifiers
for each subset an svm classifier is trained on the training subset and used to predict the corresponding leaf test test (if the leaf is not yet homogenous in terms of class labels). 

- fit_svc_linear: LinearSVC (LibLinear)
- fit_svc_sgd: SGDClassifier 
- fit_svc_kernel: tuned kernel svm

## Svm fitting and testing

In [42]:
def fit_svc_linear(subset):
    
    idx_train, X_train, y_train = subset[0][:,0], subset[0][:,1:-1], subset[0][:,-1]
    idx_test,X_test,y_test,y_tree = subset[1][:,0],subset[1][:,1:-2],subset[1][:,-2],subset[1][:,-1]
        
    # check if leaf node is heterogeneous (i.e. consists of more than one class) 
    # also check if it contains enough samples to conduct training (2)
    if len(np.unique(y_train)) >= 2 and (np.bincount(y_train.astype(int)) >= 2).all():
        
        # decide whether to solve in primal or dual
        QP_bool = False if (X_train.shape[0] > X_train.shape[0]) else True
        
        # fit svm to subset
        clf = svm.LinearSVC(class_weight='balanced', dual=QP_bool)
        clf.fit(X_train,y_train)
        
        # obtain predictions for subset
        svm_pred = clf.predict(X_test)
        svm_pred = np.concatenate((idx_test.reshape(-1,1), svm_pred.reshape(-1,1)), axis=1)
        
        return svm_pred
    
    # use tree predictions if leaf (subset) is already pure   
    else:
        tree_pred = np.concatenate((idx_test.reshape(-1,1), y_tree.reshape(-1,1)), axis=1)
        
        return tree_pred

In [55]:
def fit_svc_sgd(subset):
    
    idx_train, X_train, y_train = subset[0][:,0], subset[0][:,1:-1], subset[0][:,-1]
    idx_test,X_test,y_test,y_tree = subset[1][:,0],subset[1][:,1:-2],subset[1][:,-2],subset[1][:,-1]
        
    # check if leaf node is heterogeneous (i.e. consists of more than one class)
    if len(np.unique(y_train)) >= 2 and (np.bincount(y_train.astype(int)) >= 2).all():
        
        # fit svm to subset
        clf = linear_model.SGDClassifier(class_weight='balanced', early_stopping=True)
        clf.fit(X_train,y_train)
        
        # obtain predictions for subset
        svm_pred = clf.predict(X_test)
        svm_pred = np.concatenate((idx_test.reshape(-1,1), svm_pred.reshape(-1,1)), axis=1)
        
        return svm_pred
    
    # use tree predictions if leaf (subset) is already pure   
    else:
        tree_pred = np.concatenate((idx_test.reshape(-1,1), y_tree.reshape(-1,1)), axis=1)
        
        return tree_pred

In [94]:
def fit_svc_kernel(subset):
    
    idx_train, X_train, y_train = subset[0][:,0], subset[0][:,1:-1], subset[0][:,-1]
    idx_test,X_test,y_test,y_tree = subset[1][:,0],subset[1][:,1:-2],subset[1][:,-2],subset[1][:,-1]
    
    C_range = np.logspace(-2, 10, 13)
    gamma_range = np.logspace(-9, 3, 13)
    kernel_list = ['linear','rbf','poly']
    param_grid = dict(gamma=gamma_range, C=C_range, kernel=kernel_list)
    
    count = 0
    
    # check if leaf node is heterogeneous (i.e. consists of more than one class)
    if len(np.unique(y_train)) >= 2 and (np.bincount(y_train.astype(int)) >= 2).all():

        # fit svm to subset
        cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2)
        clf = svm.SVC(class_weight='balanced')
        tuned = RandomizedSearchCV(clf, param_distributions=param_grid, cv=cv, n_jobs=-1)
        tuned.fit(X_train, y_train)
        
        #print(tuning.best_params_)
            
        # obtain predictions for subset
        svm_pred = tuned.predict(X_test)
        svm_pred = np.concatenate((idx_test.reshape(-1,1), svm_pred.reshape(-1,1)), axis=1)
        
        return svm_pred
    
    # use tree predictions if leaf (subset) is already pure   
    else:
        tree_pred = np.concatenate((idx_test.reshape(-1,1), y_tree.reshape(-1,1)), axis=1)
        
        return tree_pred

## Concurrent execution of svms within tree
Within each decision tree, the SVMs can be trained in parallel to achieve optimal computational efficiency.

- svc_tree_linear: extratree with LinearSVC (LibLinear) in leaf node
- svc_tree_sgd: extratree with SGDClassifier in leaf node
- svc_tree_kernel: extratree with tuned kernel svm in leaf node

In [89]:
def svc_tree_linear(X_train,X_test, y_train,y_test, idx_train,idx_test):
    
    # Obtain subsets through ExtraTree partitioning
    subsets = extra_partition(X_train,X_test, y_train,y_test, idx_train,idx_test)
    
    #print("This Tree has been partitioned into {} leaf nodes".format(len(subsets)))
    
    preds_leaf = []
    
    # Run SVM's in parallel
    with Parallel() as parallel:
        result = parallel(delayed(fit_svc_linear)(subset) for subset in subsets)
        preds_leaf.append(result)
        
    # aggregate predictions of the leafs into one set of predictions for the tree
    preds_all = np.concatenate(preds_leaf[0],axis=0)
    
    # sort predictions by their index
    preds_sorted = preds_all[np.argsort(preds_all[:,0])]
    
    return preds_sorted[:,1].reshape(-1,1) # return predictions without index

In [89]:
def svc_tree_sgd(X_train,X_test, y_train,y_test, idx_train,idx_test):
    
    # Obtain subsets through ExtraTree partitioning
    subsets = extra_partition(X_train,X_test, y_train,y_test, idx_train,idx_test)
        
    preds_leaf = []
    
    # Run SVM's in parallel
    with Parallel() as parallel:
        result = parallel(delayed(fit_svc_sgd)(subset) for subset in subsets)
        preds_leaf.append(result)
        
    # aggregate predictions of the leafs into one set of predictions for the tree
    preds_all = np.concatenate(preds_leaf[0],axis=0)
    
    # sort predictions by their index
    preds_sorted = preds_all[np.argsort(preds_all[:,0])]
    
    return preds_sorted[:,1].reshape(-1,1) # return predictions without index

In [89]:
def svc_tree_kernel(X_train,X_test, y_train,y_test, idx_train,idx_test):
    
    # Obtain subsets through ExtraTree partitioning
    subsets = extra_partition(X_train,X_test, y_train,y_test, idx_train,idx_test)
        
    preds_leaf = []
    
    # Run SVM's in parallel
    with Parallel() as parallel:
        result = parallel(delayed(fit_svc_kernel)(subset) for subset in subsets)
        preds_leaf.append(result)
        
    # aggregate predictions of the leafs into one set of predictions for the tree
    preds_all = np.concatenate(preds_leaf[0],axis=0)
    
    # sort predictions by their index
    preds_sorted = preds_all[np.argsort(preds_all[:,0])]
    
    return preds_sorted[:,1].reshape(-1,1) # return predictions without index

## Serialized version

In [46]:
def svm_tree(X,y, X_train,X_test, y_train,y_test, idx_train,idx_test):
    
    subsets = get_groups(X,y, X_train,X_test, y_train,y_test, idx_train,idx_test)
    
    preds_leaf = []
    
    count = 0
    
    for subset in subsets:
        
        # Train data stored in first element of each leaf (subset)
        idx_train, X_train, y_train = subset[0][:,0], subset[0][:,1:-1], subset[0][:,-1]
        # test data  stored in second element of each leaf (subset)
        idx_test,X_test,y_test,y_tree = subset[1][:,0],subset[1][:,1:-2],subset[1][:,-2],subset[1][:,-1]
        
        # check if leaf node is heterogeneous (i.e. consists of more than one class)
        if (len(np.unique(y_train)) >= 2):
            
            # svm classifier with Radial basis function kernel
            clf = svm.SVC(kernel='rbf')
            pipe = pipeline.Pipeline([('Scaler', preprocessing.StandardScaler()), ('svc', clf)])
            # fit svm to subset
            pipe.fit(X_train,y_train)
            
            # obtain predictions for subset
            svm_pred = pipe.predict(X_test)
            # keep track of observaton indexes
            svm_pred = np.concatenate((idx_test.reshape(-1,1), svm_pred.reshape(-1,1)), axis=1)
            preds_leaf.append(svm_pred)
            
            count += 1
        
        # use tree predictions if leaf (subset) is already pure   
        else:
            tree_pred = np.concatenate((idx_test.reshape(-1,1), y_tree.reshape(-1,1)), axis=1)
            preds_leaf.append(tree_pred)
    
    print("The input space has been partitioned into {} leaf nodes, i.e. samples.".format(len(subsets)))

    print("{} of which have been used for training an SVM"\
          " given the heterogeneity of those nodes.".format(count))
    print("The remaining {} samples have the inherited"\
          " prediction from the randomized tree.".format(len(subsets)-count))
    
    # aggregate leaf predictions for all test samples across different nodes
    preds_all = np.concatenate(preds_leaf,axis=0)
    # sort predictions by their index
    preds_sorted = preds_all[np.argsort(preds_all[:,0])]
    
    return preds_sorted[:,1].reshape(-1,1) # return predictions without index

# Majority Vote
For classification, a majority vote is implemented to ultimately obtain the forest prediction, as is commonmy done in ensemble methods.

In [57]:
# Obtain majority vote for each datapoint
def majority_vote(l):
    try:
        return mode(l)
    except StatisticsError:
        return 0

## SVM embedded Forest
Finally, the above procedure can be extended to an ensemble of randomized trees with embedded SVMs. Apart from the parallel execution of the embedded svm predictors, the forest ensemble itself can also be concurrently executed. 

In [63]:
# Run trees in parallel using all cores
def rf_svc(X,y):

    forest_pred=[]
    
    X_train,X_test, y_train,y_test, idx_train,idx_test = train_test_split(X,y
                                                                          ,np.arange(len(X))
                                                                       ,test_size=1/3, stratify=y)
    
    # Rescale input space to [0,1] range (for purposes of consistency)
    scaler = preprocessing.MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    t = time.time()
    
    # Run trees in parallel
    with Parallel() as parallel:
        forest_pred = parallel(delayed(svm_parallel)(X,y 
                                                     ,X_train,X_test 
                                                     ,y_train,y_test
                                                     ,idx_train,idx_test) for i in range(1,trees))
        
    # training time
    training_time = time.time() - t
    
    # reshape array such that column k denotes prediction for tree k
    forest_pred = np.concatenate(forest_pred,axis=1)
    # majority vote
    majority = np.apply_along_axis(majority_vote, 1, forest_pred) 
    # order test set on index
    test = y_test[np.argsort(idx_test)]
    
    # accuracy
    accuracy = metrics.accuracy_score(test, majority)
    
    return accuracy, training_time